In [1]:
# @title
# MIT License

# Copyright 2024 Google LLC.
# SPDX-License-Identifier: Apache-2.0

# **Adding Looker Explore Assistant Examples**

This notebook streamlines the process of adding high-quality examples to your [Looker Explore Assistant](https://github.com/looker-open-source/looker-explore-assistant/tree/main) configuration. It allows you to upload examples from a CSV file, categorize them based on their characteristics, and format them into a clean JSON structure ready for integration.

**Key Features:**

* **CSV Upload:** Easily upload examples in a simple CSV format.
* **URL Parsing and Categorization:**  Automatically analyzes Looker Explore URLs to extract key parameters for categorization.
* **Dynamic Fields Handling:** Supports categorization of dynamic fields for flexibility.
* **JSON Output:** Generates a well-formatted JSON output that aligns with Looker's Explore Assistant requirements.

**Prerequisites:**

* **Google Colab Environment:** This notebook is designed to run in a Google Colab environment.
<!-- * **Looker API Credentials:** You'll need your Looker API credentials (client ID, client secret, base URL) to fetch query data. Store them securely in environment variables or a separate configuration file. -->

**Instructions:**

1. **Upload CSV:** Click the "Choose Files" button in the file upload section and select your CSV file containing the examples. The CSV should have two columns: `prompt` (the natural language question) and `expanded_url` (the Looker Explore URL), I recommend using [this template](https://docs.google.com/spreadsheets/d/1cTenogLjsDekwVcN-wRUgH8cLb0wdPGTNbHfXt89sS4/edit?gid=0#gid=0) to make sure it's formatted correctly.
<!-- 2. **Set Looker API Credentials:**  Replace the placeholders with your actual credentials. -->
3. **Run the Notebook:** Execute each cell in order. The final cell will generate the formatted JSON output.



In [24]:
# @title Input your GCP and LookML project variables
# @markdown You will need to override these defaults
gcp_project_id = 'data-eng-on-gcp-371519' # @param {type:"string"}
lookml_model = 'thelook' # @param {type:"string"}
lookml_explore_id = 'order_items' # @param {type:"string"}
# @markdown Do you want to include the default [Explore Assistant Examples](https://github.com/looker-open-source/looker-explore-assistant/blob/main/explore-assistant-examples/examples.json)?
Include_Defaults = "Yes"  # @param ['Yes', 'No']

In [3]:
# @title Import necessary packages
try:
  import urllib.parse
except:
  %pip install urllib.parse

try:
  import re
except:
  %pip install re

try:
  import pandas as pd
except:
  %pip install pandas

try:
  import numpy as np
except:
  %pip install numpy

try:
  import io
except:
  %pip install io

from urllib.parse import urlsplit, unquote, parse_qs
import json
import re
import pandas as pd
import numpy as np
import io
from google.colab import files


In [4]:
# @title Upload the templated CSV file with your prompts and their corresponding expanded explore URLs
# **Upload the CSV**
uploaded = files.upload()  # Opens a file upload dialog

# **Check file was uploaded**
if len(uploaded.keys()) > 0:
  for filename in uploaded.keys():
    # **Read the CSV into a Pandas DataFrame**
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
    # print("CSV looks like: ", df)
else:
  print("No CSV file was uploaded.")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Drop rows with all missing values
df.dropna(how='all', inplace=True)
df.reset_index(drop=True, inplace=True)

def parse_url(query_data):
    parsed_url = parse_qs(urlsplit(query_data).query)

    # reconstruct url for training data
    decoded_url_modifiedvisjson = ''
    # return url parameters as a string
    for query_param in parsed_url.items():
        # parse fields
        if query_param[0] == 'fields':
            decoded_url_modifiedvisjson += f'fields={query_param[1][0]}'
        # parse limit
        if query_param[0] == 'limit':
            decoded_url_modifiedvisjson += f'&limit={query_param[1][0]}'
        # parse column limit
        if query_param[0] == 'column_limit':
            decoded_url_modifiedvisjson += f'&column_limit={query_param[1][0]}'
        # parse filters
        if query_param[0].startswith('f['):
            decoded_url_modifiedvisjson += f'&{query_param[0]}={query_param[1][0]}'
        # parse pivots
        if query_param[0] == 'pivots':
            decoded_url_modifiedvisjson += f'&pivots={query_param[1][0]}'
        # parse fill fields
        if query_param[0] == 'fill_fields':
            decoded_url_modifiedvisjson += f'&fill_fields={query_param[1][0]}'
        # parse dynamic fields ie. custom fields and table calcs
        if query_param[0] == 'dynamic_fields':
            decoded_url_modifiedvisjson += f'&dynamic_fields={query_param[1][0]}'
        # parse sorts
        if query_param[0] == 'sorts':
            decoded_url_modifiedvisjson += f'&sorts={query_param[1][0]}'
        # parse just vis type
        if query_param[0] == 'vis':
            vis_type = re.search(r'("type":\s*"([^,}]+))', query_param[1][0])
            if vis_type:
                decoded_url_modifiedvisjson += '&vis={' + vis_type.group(1) + '}'
            # if no vis type, don't add
            else:
                continue
        # if none of the above skip as it's not needed
        else:
            continue
    return decoded_url_modifiedvisjson
    # print(f"""Components of your url: \n {parsed_url}\nReconstructed url for Explore Assistant (copy this): \n {decoded_url_modifiedvisjson}""")

if 'formatted_url' not in df.columns:
    df['formatted_url'] = ''


for index, row in df.iterrows():
  formatted_url = parse_url(row['expanded_url'])
  df.loc[index, 'formatted_url'] = formatted_url



Saving [Template] Looker Explore Assistant New Example Template - Sheet1 (7).csv to [Template] Looker Explore Assistant New Example Template - Sheet1 (7).csv


In [25]:
# @title Generate examples
# @markdown Copy the resulting code below.

# @markdown If you are using the BQML deployment, paste the code into your BQ console in the project where you deployed the explore assistant and then run all sections of the code.

# @markdown If you are using the cloud functions deployment, add the formatted examples to the jsonl file within your explore assistant cloud function.
# def escape_quotes(text):
#     """Escapes all double quotes in a string by adding a backslash before them."""
#     return text.replace('"', '\\"')

# examples = ""
# examples += '[{'

# for index, row in df.iterrows():
#   row[0] = escape_quotes(row[0])
#   row[2] = escape_quotes(row[2])
#   if index == len(df) - 1:
#     examples += f'\n"input": "{row["prompt"]}",\n'
#     examples += f'"output": "{row["formatted_url"]}"\n'+'}]\n'
#   else:
#     examples += f'\n"input": "{row["prompt"]}",\n'
#     examples += f'"output": "{row["formatted_url"]}"\n'+'}, {'

# examples = examples.strip()  # Remove leading/trailing whitespace
# # examples = re.findall(r'{.*?}', examples)
# examples = json.loads(examples)
examples = []
for _, rows in df.iterrows():
  examples.append({
    "input": rows["prompt"],
    "output": rows["formatted_url"]
  })

# print(json.dumps(examples))
if Include_Defaults == "Yes":
  import requests
  url = "https://raw.githubusercontent.com/looker-open-source/looker-explore-assistant/main/explore-assistant-examples/examples.json"
  response = requests.get(url)
  response.raise_for_status()  # Check for errors

  # Load the JSON data into a Python dictionary or list
  examples_data = response.json()
  final_list = examples + examples_data
  final_list = json.dumps(final_list)
  add_example = add_example = f"""SET additional_examples = \"\"\"{final_list}\"\"\";"""
else:
  examples = json.dumps(examples)
  add_example = add_example = f"""SET additional_examples = \"\"\"{examples}\"\"\";"""


setup = "DECLARE additional_examples STRING;"

delete_query = f"DELETE FROM `{gcp_project_id}.explore_assistant.explore_assistant_examples` WHERE explore_id = '{lookml_model}:{lookml_explore_id}';"

insert_query = f"INSERT INTO `{gcp_project_id}.explore_assistant.explore_assistant_examples` (explore_id, examples) \
VALUES ('{lookml_model}:{lookml_explore_id}', TRIM(additional_examples))"


print(setup, "\n")

print(add_example, "\n")

print(delete_query,"\n")

print(insert_query)




DECLARE additional_examples STRING; 

SET additional_examples = """[{"input": "what is my total gross margin broken out by day for the last 90 days", "output": "fields=order_items.total_gross_margin,order_items.created_date&fill_fields=order_items.created_date&f[order_items.created_date]=90 days&limit=500&column_limit=50"}, {"input": "total inventory by age", "output": "fields=inventory_items.days_in_inventory,inventory_items.count&sorts=inventory_items.days_in_inventory&limit=500&column_limit=50"}, {"input": "total gross sales by date for the last 30 days, pivoted by country as a line chart", "output": "fields=order_items.created_date,order_items.total_sale_price,users.country&pivots=users.country&fill_fields=order_items.created_date&f[order_items.created_date]=30 days&sorts=users.country,order_items.created_date desc&limit=500&column_limit=50&vis={\"type\":\"looker_line\"}"}, {"input": "total sale for last 30 days broken out by day ", "output": "fields=order_items.created_date,order_